In [1]:
import numpy as np
import os
from sklearn.model_selection import KFold
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from tensorflow.keras.layers import Dense , Dropout
from scikeras.wrappers import KerasRegressor 
from tensorflow.keras.callbacks import EarlyStopping
import time
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras_tuner import RandomSearch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import GridSearchCV

# Einlesen der Pickle-Dateien und Vorverarbeitung des Inhaltes

In [2]:
data = pd.read_pickle('C:/Users/erikm/Desktop/Diplomarbeit Erik Marr/Daten/Finish/Finish_ALL_D3_500_I_F_PKL.pkl')

data

,X-Koordinate,Y-Koordinate,Zeitpunkt,Strom,Kraft,Temperatur
254520,0.00000,-0.00200,500,6000,5000,7.535400e+02
254521,0.00000,-0.00196,500,6000,5000,7.936300e+02
254522,0.00000,-0.00192,500,6000,5000,8.356500e+02
254523,0.00000,-0.00188,500,6000,5000,8.775000e+02
254524,0.00000,-0.00184,500,6000,5000,9.199800e+02
...,...,...,...,...,...,...
2087059,0.00248,0.00184,500,9000,5000,1.110600e+03
2087060,0.00248,0.00188,500,9000,5000,1.046600e+03
2087061,0.00248,0.00192,500,9000,5000,9.810900e+02
2087062,0.00248,0.00196,500,9000,5000,7.888600e-31


In [3]:
# Mit der Bedingung wird die Simulationsdatei I7000 F6000 ausgewählt.
bedingung = (data['Kraft'] == 6000) & (data['Strom'] == 7000)
df_test = data[bedingung].copy()

print(df_test)

data_all = data.drop(df_test.index) #Simualtionsdatei aus den Trainingsdaten löschen

print(data_all[(data_all['Kraft'] == 6000) & (data_all['Strom'] == 7000)]) #Kontrolle, ob Simulationsdatei wirklich gelöscht ist
df_test

         X-Koordinate  Y-Koordinate  Zeitpunkt  Strom  Kraft  Temperatur
1037169       0.00000      -0.00200        500   7000   6000      811.76
1037170       0.00000      -0.00196        500   7000   6000      853.27
1037171       0.00000      -0.00192        500   7000   6000      897.57
1037172       0.00000      -0.00188        500   7000   6000      941.21
1037173       0.00000      -0.00184        500   7000   6000      986.34
...               ...           ...        ...    ...    ...         ...
1043527       0.00248       0.00184        500   7000   6000      784.55
1043528       0.00248       0.00188        500   7000   6000      745.87
1043529       0.00248       0.00192        500   7000   6000      706.17
1043530       0.00248       0.00196        500   7000   6000      693.28
1043531       0.00248       0.00200        500   7000   6000      687.80

[6363 rows x 6 columns]
Empty DataFrame
Columns: [X-Koordinate, Y-Koordinate, Zeitpunkt, Strom, Kraft, Temperatur]
Index: [

,X-Koordinate,Y-Koordinate,Zeitpunkt,Strom,Kraft,Temperatur
1037169,0.00000,-0.00200,500,7000,6000,811.76
1037170,0.00000,-0.00196,500,7000,6000,853.27
1037171,0.00000,-0.00192,500,7000,6000,897.57
1037172,0.00000,-0.00188,500,7000,6000,941.21
1037173,0.00000,-0.00184,500,7000,6000,986.34
...,...,...,...,...,...,...
1043527,0.00248,0.00184,500,7000,6000,784.55
1043528,0.00248,0.00188,500,7000,6000,745.87
1043529,0.00248,0.00192,500,7000,6000,706.17
1043530,0.00248,0.00196,500,7000,6000,693.28


In [4]:
data_all = data_all.drop('Zeitpunkt', axis = 1) #Zeitpunkt löschen, weil immer 500ms 
data_all

,X-Koordinate,Y-Koordinate,Strom,Kraft,Temperatur
254520,0.00000,-0.00200,6000,5000,7.535400e+02
254521,0.00000,-0.00196,6000,5000,7.936300e+02
254522,0.00000,-0.00192,6000,5000,8.356500e+02
254523,0.00000,-0.00188,6000,5000,8.775000e+02
254524,0.00000,-0.00184,6000,5000,9.199800e+02
...,...,...,...,...,...
2087059,0.00248,0.00184,9000,5000,1.110600e+03
2087060,0.00248,0.00188,9000,5000,1.046600e+03
2087061,0.00248,0.00192,9000,5000,9.810900e+02
2087062,0.00248,0.00196,9000,5000,7.888600e-31


In [5]:
# Shufflen der Daten
data_all = data_all.sample(frac=1, random_state=42)  # Hier wird 42 als Random State verwendet, um die Ergebnisse reproduzierbar zu machen
df_reset = data_all.reset_index(drop=True)
df_reset

,X-Koordinate,Y-Koordinate,Strom,Kraft,Temperatur
0,0.00112,-0.00064,6000,6000,1617.10
1,0.00108,-0.00036,8000,7000,2058.20
2,0.00136,0.00096,6000,6000,1420.30
3,0.00064,0.00028,6000,5000,1829.90
4,0.00048,0.00188,6000,6000,693.83
...,...,...,...,...,...
44536,0.00248,-0.00188,6000,5000,842.80
44537,0.00192,0.00092,6000,6000,1342.80
44538,0.00248,0.00124,8000,7000,1434.90
44539,0.00032,0.00008,6000,5000,1873.40


In [6]:
#Zuweisung der Trainingsdaten
y = df_reset["Temperatur"]
# Korrektur: Verwenden Sie den Spaltennamen direkt, ohne Indexierung der columns-Eigenschaft
X = df_reset.drop("Temperatur", axis=1)

print(X[(X['Kraft'] == 6000) & (X['Strom'] == 7000)]) # Erneute Kontrolle

Empty DataFrame
Columns: [X-Koordinate, Y-Koordinate, Strom, Kraft]
Index: []


In [7]:
#Zuweisung der Testdaten
label = df_test["Temperatur"]
# Korrektur: Verwenden Sie den Spaltennamen direkt, ohne Indexierung der columns-Eigenschaft
X_2 = df_test.drop("Temperatur", axis=1)
X_2 = X_2.drop('Zeitpunkt',axis =1)
X_2 = X_2.reset_index(drop=True)
y_2 = label

In [8]:
X_2

,X-Koordinate,Y-Koordinate,Strom,Kraft
0,0.00000,-0.00200,7000,6000
1,0.00000,-0.00196,7000,6000
2,0.00000,-0.00192,7000,6000
3,0.00000,-0.00188,7000,6000
4,0.00000,-0.00184,7000,6000
...,...,...,...,...
6358,0.00248,0.00184,7000,6000
6359,0.00248,0.00188,7000,6000
6360,0.00248,0.00192,7000,6000
6361,0.00248,0.00196,7000,6000


In [9]:
y

0        1617.10
1        2058.20
2        1420.30
3        1829.90
4         693.83
          ...   
44536     842.80
44537    1342.80
44538    1434.90
44539    1873.40
44540    2010.50
Name: Temperatur, Length: 44541, dtype: float64

In [10]:
# Initialisiere einen MinMaxScaler für die Features
scaler_features = MinMaxScaler()
# Skaliere X_train und X_test
X_train_scaled = scaler_features.fit_transform(X)
X_test_scaled = scaler_features.transform(X_2)  # Nutze gleiche Skalierungsparameter ohne das X_Test Informationen einfließen

# Initialisiere einen SEPARATEN MinMaxScaler für das Ziel
scaler_target = MinMaxScaler()
# Skaliere y_train und y_test. Beachte, dass y_train.reshape(-1, 1) verwendet wird, da MinMaxScaler 
# erwartet, dass die Eingaben als 2D-Arrays kommen, und Ziele normalerweise als 1D-Arrays vorliegen.
y_train_scaled = scaler_target.fit_transform(y.values.reshape(-1, 1))
y_test_scaled = scaler_target.transform(y_2.values.reshape(-1, 1))

In [11]:
X_test_scaled

array([[0.        , 0.        , 0.33333333, 0.25      ],
       [0.        , 0.01      , 0.33333333, 0.25      ],
       [0.        , 0.02      , 0.33333333, 0.25      ],
       ...,
       [1.        , 0.98      , 0.33333333, 0.25      ],
       [1.        , 0.99      , 0.33333333, 0.25      ],
       [1.        , 1.        , 0.33333333, 0.25      ]])

In [13]:
y_train_scaled.max()

1.0

In [15]:
# Netzwerkarchitektur
model = Sequential([

    Dense(136, activation='relu', input_shape=(4,), kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001)),

    Dense(216, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001)),
    
    Dense(104, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001)),
    
    Dense(328, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001)),
    
    Dense(8, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001)),
    
    Dense(120, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001)),
    
    Dense(1 , activation = 'linear')
])

# Optimierer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)  #0.0001

# Modell kompilieren (Verwendung von mean_squared_error als Verlustfunktion für Regression)
model.compile(optimizer=optimizer,
              loss='mean_squared_error',
              metrics=['mae'])  # Metriken für Regression: Mean Absolute Error und Mean Squared Error

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min', restore_best_weights=True)

# Trainingsparameter
batch_size = 25   #25
epochs = 1000

# Modell trainieren (Annahme: X_train, y_train, X_val, y_val sind vordefiniert)
history = model.fit(X_train_scaled, y_train_scaled,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2,
                    callbacks=[early_stopping])

model.save('D3_I_F_1.h5')

Epoch 1/1000
1426/1426 [==============================] - 3s 2ms/step - loss: 0.0476 - mae: 0.0996 - val_loss: 0.0151 - val_mae: 0.0194
Epoch 2/1000
1426/1426 [==============================] - 2s 2ms/step - loss: 0.0140 - mae: 0.0138 - val_loss: 0.0131 - val_mae: 0.0090
Epoch 3/1000
1426/1426 [==============================] - 3s 2ms/step - loss: 0.0127 - mae: 0.0105 - val_loss: 0.0123 - val_mae: 0.0113
Epoch 4/1000
1426/1426 [==============================] - 2s 2ms/step - loss: 0.0118 - mae: 0.0099 - val_loss: 0.0113 - val_mae: 0.0090
Epoch 5/1000
1426/1426 [==============================] - 2s 2ms/step - loss: 0.0110 - mae: 0.0094 - val_loss: 0.0105 - val_mae: 0.0068
Epoch 6/1000
1426/1426 [==============================] - 3s 2ms/step - loss: 0.0102 - mae: 0.0089 - val_loss: 0.0098 - val_mae: 0.0094
Epoch 7/1000
1426/1426 [==============================] - 3s 2ms/step - loss: 0.0095 - mae: 0.0084 - val_loss: 0.0092 - val_mae: 0.0089
Epoch 8/1000
1426/1426 [========================

C:\Users\erikm\Desktop\Diplomarbeit Erik Marr\Projekt X\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
model = tf.keras.models.load_model('Trainierte_Netze/D3_I_F_1.h5')

In [16]:
# Initialisiere Listen, um Ergebnisse zu speichern
val_loss_results = []
val_mae_results = []

# Funktion, um das Modell zu erstellen
def create_model():
    model = Sequential([
            Dense(136, activation='relu', input_shape=(4,), kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001)),

            Dense(216, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001)),

            Dense(104, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001)),

            Dense(328, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001)),

            Dense(8, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001)),

            Dense(120, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001)),

            Dense(1 , activation = 'linear')

    ])
    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
    return model

# K-Fold Cross-Validation Konfiguration
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Leistungsüberwachung
fold_no = 1
for train_index, val_index in kf.split(X_train_scaled):
    X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[val_index]
    y_train_fold, y_val_fold = y_train_scaled[train_index], y_train_scaled[val_index]

    model = create_model()

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min', restore_best_weights=True)

    print(f'Training für Fold {fold_no}...')
    history = model.fit(X_train_fold, y_train_fold, batch_size=25, epochs=1000, validation_data=(X_val_fold, y_val_fold), callbacks=[early_stopping], verbose=1)

    # Speichere die Ergebnisse des aktuellen Folds
    val_loss_results.append(min(history.history['val_loss']))
    val_mae_results.append(min(history.history['val_mae']))

    fold_no += 1

# Berechne den Durchschnitt über alle Folds
average_val_loss = np.mean(val_loss_results)
average_val_mae = np.mean(val_mae_results)
# 
# Umwandeln der Listen in Pandas DataFrames
val_loss_df = pd.DataFrame(val_loss_results, columns=['Validation Loss'])
val_mae_df = pd.DataFrame(val_mae_results, columns=['Validation MAE'])

# Speichern der DataFrames in CSV-Dateien
val_loss_df.to_csv('val_loss_results_D3_I_F_1.csv', index=False)
val_mae_df.to_csv('val_mae_results_D3_I_F_1.csv', index=False)

# Gib die durchschnittlichen Ergebnisse aus
print(f'Durchschnittlicher Validation Loss: {average_val_loss}')
print(f'Durchschnittlicher Validation MAE: {average_val_mae}')


In [17]:
#Auswertung auf dem Testdatensatz
results = model.evaluate(X_test_scaled, y_test_scaled, verbose=2)
results

199/199 - 1s - loss: 2.0120e-04 - mae: 0.0029 - 514ms/epoch - 3ms/step


[0.00020120339468121529, 0.002875363687053323]

In [18]:
#Berechnung des MAE und MAPE
scaled_predicted_values = model.predict(X_test_scaled, verbose = 0)

# Führen Sie die Rücktransformation der skalierten Werte durch
original_predicted_values = scaler_target.inverse_transform(scaled_predicted_values)
original_actual_values = scaler_target.inverse_transform(y_test_scaled)  # y_test sind die skalierten tatsächlichen Werte
print(f' Bsp. Predicted: {original_predicted_values[100]} Actual: {original_actual_values[100]} ')

def calculate_mae(list1, list2):
    # Stelle sicher, dass beide Listen die gleiche Länge haben
    if len(list1) != len(list2):
        raise ValueError("Listen müssen die gleiche Länge haben")

    # Berechne die absolute Differenz zwischen den Elementen der Listen
    differences = [abs(x - y) for x, y in zip(list1, list2)]

    # Berechne den Durchschnitt der absoluten Differenzen
    mae = sum(differences) / len(differences)

    return mae

# Beispiel
list1 = original_predicted_values
list2 = original_actual_values

mae = calculate_mae(list1, list2)
print(f"Durchschnittliche Abweichung (MAE): {mae}")

errors = np.abs((original_actual_values - original_predicted_values) / original_actual_values)
mape = np.mean(errors) * 100
print(mape)

 Bsp. Predicted: [721.09924] Actual: [735.96] 
Durchschnittliche Abweichung (MAE): [7.41038792]
0.5500216618740962


In [19]:
# R^2 Berechnung
def calculate_r_squared(predicted, actual):
    # Berechnung des Mittelwerts der tatsächlichen Werte
    mean_actual = sum(actual) / len(actual)
    
    # Berechnung der totalen Summe der Quadrate (SST)
    sst = sum((x - mean_actual) ** 2 for x in actual)
    
    # Berechnung der Summe der Quadrate der Residuen (SSE)
    sse = sum((actual[i] - predicted[i]) ** 2 for i in range(len(actual)))
    
    # Berechnung des R^2-Wertes
    r_squared = 1 - (sse / sst)
    
    return r_squared

# Berechnung von R^2 mit den bereitgestellten Listen
r_squared = calculate_r_squared(list1, list2)

print(f"R^2: {r_squared}")

R^2: [0.99918974]


In [20]:
#Überprüfung der maximalen Unterschiede zwischen Echt und Vorhergesagt
df_result = pd.DataFrame({'Echt': [val[0] for val in list2], 'Temperatur': [val[0] for val in list1]})
df_result['X-Koordinate'] = X_test_scaled[:, 0]
df_result['Y-Koordinate'] = X_test_scaled[:, 1]
df_result['Differenz'] = abs(df_result['Echt'] - df_result['Temperatur'])
df_result['Differenz'].sort_values()
sorted_df = df_result.sort_values(by= 'Differenz')
Anzahl_Punkte = (sorted_df['Differenz'] > 20).sum()
print("Anzahl der Werte die kleiner sind:", Anzahl_Punkte)

sorted_df

Anzahl der Werte die kleiner sind: 514


,Echt,Temperatur,X-Koordinate,Y-Koordinate,Differenz
1358,1981.0,1980.999390,0.209677,0.45,0.000610
1431,1519.8,1519.797241,0.225806,0.17,0.002759
3675,1858.6,1858.595947,0.580645,0.39,0.004053
5739,1287.7,1287.704468,0.903226,0.83,0.004468
661,1975.7,1975.707397,0.096774,0.55,0.007397
...,...,...,...,...,...
6325,1613.1,1574.510498,1.000000,0.63,38.589502
6295,1606.5,1567.853394,1.000000,0.33,38.646606
6324,1623.3,1584.372925,1.000000,0.62,38.927075
6297,1627.0,1588.036499,1.000000,0.35,38.963501


In [18]:
#Speichern der Vorhersage
# df_test['Vorhergesagt'] = list1
# df_test.to_pickle('C:/Users/erikm/Desktop/Diplomarbeit Erik Marr/Daten/Finish/Finish_I7000_F6000_D3_500_I_F_PKL_Prediction.pkl')

In [21]:
#Trainingsverlauf
mse = history.history['loss']
val_mse = history.history['val_loss']

epochs = range(1, len(mse) + 1)

plt.figure(figsize=(10, 6))
plt.plot(epochs, mse, 'r', label='Training MSE')
plt.plot(epochs, val_mse, 'b', label='Validation MSE')
plt.title('Training and Validation MSE')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()
plt.show()

NameError: name 'history' is not defined

# GridSearch

In [49]:
def build_model(learning_rate=0.001, activation='relu', regularization=0.0001, dropout_rate=0.0):
    model = Sequential()
    model.add(Dense(136, activation=activation, input_shape=(4,), kernel_initializer='he_uniform', kernel_regularizer=l2(regularization)))
    model.add(Dropout(dropout_rate))

    model.add(Dense(216, activation=activation, kernel_initializer='he_uniform', kernel_regularizer=l2(regularization)))
    model.add(Dropout(dropout_rate))

    model.add(Dense(104, activation=activation, kernel_initializer='he_uniform', kernel_regularizer=l2(regularization)))
    model.add(Dropout(dropout_rate))

    model.add(Dense(328, activation=activation, kernel_initializer='he_uniform', kernel_regularizer=l2(regularization)))
    model.add(Dropout(dropout_rate))

    model.add(Dense(8, activation=activation, kernel_initializer='he_uniform', kernel_regularizer=l2(regularization)))
    model.add(Dropout(dropout_rate))

    model.add(Dense(120, activation=activation, kernel_initializer='he_uniform', kernel_regularizer=l2(regularization)))
    model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error', metrics=['mae'])
    return model

model = KerasRegressor(model=build_model, verbose=2)

# Anpassung der Parameter im param_grid
param_grid = {
    'model__learning_rate': [0.01, 0.001, 0.0001],
    'model__regularization': [0.001, 0.0001, 0.00001],
    'fit__batch_size': [25, 50, 75, 100],
    'fit__epochs': [50],
    'model__dropout_rate' : [0.0, 0.1, 0.2]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=2)
# Hinweis: Sicherstellen, dass Daten (X_train_scaled, y_train_scaled) korrekt definiert sind
grid_result = grid_search.fit(X_train_scaled, y_train_scaled)
# Beste Parameter und Score ausgeben
print("Beste Parameter:", grid_search.best_params_)
print("Beste Genauigkeit:", grid_search.best_score_)

with open("Gridsearch_D3_I_F_1.txt", "w") as f:
    f.write(f"Beste Parameter: {grid_search.best_params_}\n")
    f.write(f"Beste Genauigkeit: {grid_search.best_score_}")


# Random Search

In [50]:
# Funktion zum Erstellen des Modells
def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('input_units', min_value=8, max_value=328, step=16), input_shape=(4,), activation='relu'))
    for i in range(hp.Int('n_layers', 1, 10)):
        model.add(Dense(hp.Int(f'units_{i}', min_value=8, max_value=328, step=16), activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Durchführung der Random Search dreimal
for run in range(1, 4):
    # Anpassen des Verzeichnisses und des Projektnamens für jeden Durchlauf
    directory = 'random_search'
    project_name = f'random_search_D3_I_F_1_{run}'

    tuner = RandomSearch(
        build_model, 
        objective='val_loss',
        max_trials=100,
        executions_per_trial=1,
        directory=directory,
        project_name=project_name
    )

    # Durchführung des Random Search
    tuner.search(X_train_scaled, y_train_scaled, epochs=50, verbose =0, batch_size=50, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

    # Abrufen und Speichern des besten Modells
    best_model = tuner.get_best_models(num_models=1)[0]
    model_path = os.path.join(directory, project_name, 'best_model.h5') 
    best_model.save(model_path)


    # Optional: Abrufen und Ausgeben der besten Hyperparameter
    best_hyperparameters = tuner.get_best_hyperparameters()[0]

    # Konvertieren der Hyperparameter in ein DataFrame
    df_hyperparameters = pd.DataFrame([best_hyperparameters.values])
    # Speichern des DataFrame als CSV
    df_hyperparameters.to_csv(f'random_search_D3_I_F_1_{run}.csv', index=False)

    print(f"Beste Hyperparameter für Lauf {run}: {best_hyperparameters.values}")